# Journal de bord

Copier-coller les cellules comme canevas pour y remplir les informations.

## 📅 Journal de bord – Jour 1 (17/12/2025)

### 🔧 Étapes / Actions réalisées
- Découverte du sujet,
- Récupération des ressources,
- Structuration des dossiers
- Mise en place du tracking **git**

**Première approche de découpage des tâches**

**Traitement des fichiers de données tabulaires**
1. Analyser les fichiers de données et définir la stratégie de nettoyage préalable : données sensibles, données personnelles, données interdites,
2. Procéder à la fusion des jeux de données en incluant une information supplémentaire pour ne pas perdre l'origine de l'information
3. Produire dans outputs/data_processed le fichier csv résultant

## 📅 Journal de bord – Jour 2 (18/12/2025)

### 🔧 Étapes / Actions réalisées

- Mise en attente des processus de détection des NaN et Outliers, j'ajouterai les handlers associés plus tard en mettant à jour le pipeline,
- Pour faciliter l'entraînement des modèles selon : deux algorithmes et 4 scénarios, je vais mettre en place un pattern *Strategy* ; concrètement ce pattern me permettra de rejouer le même handler en changeant dynamiquement la stratégie d'entraînement. Le principe DRY (Don't Repeat Yourself) oblige à réfléchir à la manière de ne pas réécrire plusieurs fois la même chose,
- Les deux stratégies : Logistic Regression (LR) et Random Forest (RF)

**Note** : Intégration de MLFlow et des artefacts, paramètres (hyper paramètres) qui permettra de choisir le modèle le plus performant et facilitant la lecture des performances globales.
- Matrice de confusion (quelque soit le modèle),
- Matrice de corrélation (quelque soit le modèle),
- Courbe ROC (Receiver Operating Characteristic) : explicabilité et proposition d'ajustement,
- Precision-Recall pour la régression uniquement

**Retour sur Outliers Detection et gestion des données manquantes**

Deux approches différents pour la gestion des valeurs abérantes : 
- IQR permettant de détecter facilement les valeurs hors bornes,
- Isolation Forest : plus intelligent, IF permet de détecter des valeurs hors normes sur l'ensemble des données numériques.
Afin de gérer au mieux la détection des outliers, j'ai implémenté le deux stratégies, me permettant aisément de switcher de l'une à l'autre pour affiner l'entraînement ultérieur et éviter trop de dispersion.

Concernant l'imputation, là encore, afin d'affiner les données avant entraînement :
- Remplacement par valeur médiane qui me permettra de conserver un nombre de lignes suffisant pour assurer la pertinence de l'entraînement. Attention cependant à ce que ce "remplacement" ne sur représente pas une classe,
- Suppression de ligne : le risque étant de sous-représenter une classe
La stratégie pourra toujours être remplacée après les premiers entraînement et l'analyse des métriques. On partira dans un premier temps sur *MedianStrategy*

Au cours de ma réflexion, plus généralement sur les *imputations*, le traitement par colonnes "identifiées" par l'humain peut conduire à des erreurs, notamment si le scientist n'a pas la connaissance métier suffisante.

J'ai décidé d'expérimenter un processus "regressif" pour estimer automatiquement les colonnes qui doivent être imputées : *AIImputer*

## 📅 Journal de bord – Jour 3 (23/12/2025)

### 📝 Observations / Difficultés

Après les premiers tests de *nettoyage* du dataset, il s'avère que je dois corriger des erreurs de transmission du dataframe dans le contexte d'éxécution.

- Corrections majeures sur les copies du dataframe à chaque étape pour éviter de perdre la colonne de référence

### ✅ Prochaines étapes

Ma réflexion me conduit à envisager de définir les différents scenario et la sélection des stratégies dans un fichier **yaml** en amont de l'exécution. Cette approche permettra de pré-scripter les stratégies à utiliser pour le nettoyage et l'entraînement.
Les avantanges de cette approche :
- créer un worflow plus "industriel" respectant le *Open/Closed Principle*, facilitant la compréhension du **pipeline** sans avoir à lire le code,
- Flexibilité : il est possible de changer les stratégies sans toucher au code lui même,
- Versioning : Il est beaucoup plus simple de versionner le fichier de configuration et de rejouer l'approche la plus pertinente,
- Scalable : on peut ajouter autant de scénarii en les ajoutants à la configuration yaml

Je dois commencer à réfléfchir à l'intégration du processus de CI/CD, la conception des images Docker et le déploiement du modèle final pour être exploité par une API.

**API**
- Prévoir un Dockerfile pour l'API (FastAPI),
- Gérer les tests unitaires sur les endpoints (GET /, GET /health, POST /predict)
- Intégrer un "middleware" pour une authentication plus musclée

**Client**
- Créer une interface simple (streamlit) pour procéder à l'authentification (basic) et fournir un formulaire destiné à la prédiction

**Docker**
- Créer un docker-compose pour charger les services (front et back),
- Créer un Dockerfile pour l'API

**CI/CD**
- Assurer la portabilité : Github / Gitlab
- Définir les "steps"
- Logger dans Discord le résultat des étapes

### 📝 Observations / Difficultés
- Ne pas oublier de transmettre les paramètres corrects aux Handlers pour permettre les instanciations correctes des classes métiers

## 📅 Journal de bord – Jour 4 (24/12/2025)

1. Le pipeline configuré nécessite une intervention technique pour être mis à jour (algorithmes, métriques, ...).

L'idée générale est de permettre à l'utilisateur final de modifier / tester différentes configuration avant de pouvoir procéder à un nouvel entraînement, ainsi que de versionner les différentes configurations utilisées pour pouvoir réentraîner le cas échéant.

2. l'utilisateur doit pouvoir choisir le modèle à utiliser pour ses prédictions :

- **accuracy** : s'il souhaite le maximum de précision, au risque de laisser passer des élèves au travers des mailles du filet (implique un risque de voir se dégrader les résulats généraux de l'établissement, une notoriété qui décroît, ...),
- **auc** : s'il souhaite râtisser plus large, au risque d'inclure des élèves qui ne sont pas concernés (implique un risque financier pour les cours de soutien, de recrutement de professeurs, de mise à disposition de salles, de créneaux, ...)

### 📝 Observations / Difficultés
- Il est nécessaire de refactorer le code initial : extraire les handlers du notebook principal pour pouvoir exploiter la chaîne de puis l'API à la demande du *Client*,
- Partir sur la création d'un dossier "*core*" pour regrouper les classes de manière cohérente

## 📅 Journal de bord – Jour 5 (25/12/2025)

1. Retester après refactorage des packages
Le refctorage entraîne des effets de bords, un test complet doit permettre de regénérer les modèles, en y intégrant un "*shuffle*" du dataframe pour éviter *l'overfitting* en réentraînant systématiquement sur la même base de prédictions / entraînements.

2. Déploiement CI/CD
Après l'exploitation via le Notebook et le refactorage, l'intégration de l'API et du Frontend, il est temps de valider l'outillage Docker (valider Dockerfile et Docker Compose)

Intégrer les deux workflows : github et gitlab

Monitorer via Prométhéus et Grafana



## 📅 Journal de bord – Jour 6 (29/12/2025)

1. Préparation des images Docker
Après refactorage, deux images Docker ont été créées :
- backend,
- frontend

2. Test en environnement Docker
Tests de communication frontend - backend, contrôle du flux Request / Response. Mises au point en mode exploitation.



### 📝 Observations / Difficultés

La construction du projet en mode Notebook a entraîné beaucoup de refactorage (notamment de changement de dossier / module). A l'avenir, il est préférable de commencer par une architecture plus solide et finir par une exploitation Notebook :

- Eviter de copier un ensemble inutile dans les images Docker,
- Eviter les confusions d'imports,
- Faciliter les mises à jour

Les premières mises à jour à réaliser :
- Stockage des données d'entraînement joblib pour éviter les erreurs de fitting lors des prédictions

### ✅ Prochaines étapes
- Valider le jeu de données après avoir créé le premier pipeline pour mise en conformité **RGPD** et **IAAct**. Cette étape permettra de rendre le modèle de prédiction conforme en éliminant, minimisant, anonymisant les données traitées,
- Gérer l'exclusion des données lors de l'étape "sensitive data handler",
- Ajouter une étape d'anonymisation et minimisation dans le pipeline, prouvant l'efficacité du pattern *Chain of Responsibilities* en intégrant simplement un nouvel handler dans la chaîne et sans toucher le reste du code.

## 📅 Journal de bord – Jour 7 (09/01/2026)

Suite à une manipulation GIT risquée, je dois repartir sur un clone du dépôt existant.

**A noter** : Nettoyer le dépôt pour ne plus tenir compte des artefacts générés (fichiers intermédiaires, graphes, courbes)

**Intégrations**
- Feature Importance Plot : ce graphe a pour but d'expliquer quelles sont les variables qui ont le plus d'influence sur la prédiction. Pour un algorithme Random Forest, ce graphe illustre donc les données réellement importantes, permettant ainsi d'adapter le fichier de données en éliminant les variables avec une importance faible, sans dégrader la prédiction,
- Precision-Recall Curve : Cette courbe évalue la capacité du modèle à identifier précisément les élèves en risque d'échec en minimisant les faux positifs. La précision mesure le ration entre élèves effectivement classés en échec parmi toutes les prédictions réalisées. Le Recall quand à lui mesure la proportion des élèves réellement en échec que le modèle a réussi à détecter.
- One-Tree : Aide à comprendre (explicabilité) comment le modèle prend ses décisions, en passant à travers les noeuds contenant une condition.

**Mise en oeuvre**

- Ajouter l'étape de minimisation, anonymisation pour assurer la conformité règlementaire,
- Intégrer l'étape dans le pipeline,
- Réentraîner le modèle après traitement.

**Analyse approfondie du jeu de données**

| Catégorie | Données concernées |Statut Réglementaire |Recommandation / Action |
| --------- | ------------------ | ------------------- | ---------------------- |
|Données Autorisées | studytime, failures, absences, G1, G2, traveltime, school| Faible risque ; Directement utilisables. Ce sont des données purement académiques ou de logistique scolaire.|
| Données Sensibles (Risque de Discrimination)| sex ; age ; address (U/R) ; famsize |Haute Surveillance|Le AI Act interdit l'utilisation de critères discriminatoires pour le scoring. sex et address ne devraient être utilisés que pour vérifier l'absence de biais (Equité). |
| Données de Vie Privée (Socio-éco) | Mjob ; Fjob ; Medu ; Fedu ; Pstatus ; guardian | Anonymisation / Agrégation. Ces données révèlent le statut social. Sous le RGPD elles doivent être traitées avec le principe de ""minimisation""." |
| Données Comportementales / Santé |Dalc ; Walc (alcool) ; health ; romantic | Données Sensibles (Art. 9 RGPD) | "Interdit ou restreint. Les données de santé et les habitudes de consommation sont ""spéciales"". Leur usage pour prédire un échec scolaire est très risqué juridiquement."|
| Identifiants Indirects | Combinaison de school + age + Mjob + Fjob | Risque de Ré-identification |Pseudonymisation requise. Une combinaison précise de ces facteurs peut permettre d'identifier un élève spécifique dans une petite école.|
